In [1]:
import pandas as pd
import numpy as np
import pickle as pk
import re

import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

In [2]:
case_fraud = pd.read_csv('../1_Data/case_fraud.csv')
case_fraud.head(1)

,index,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,사건종류코드,판결유형,선고,판례상세링크,판시사항,판결요지,참조조문,참조판례,판례내용
0,20,238021,특정경제범죄가중처벌등에관한법률위반(사기)·사기,2023도12424,2023.11.16,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=yeorii&target=prec&ID=23...,"유사수신행위를 금지·처벌하는 유사수신행위의 규제에 관한 법률 제6조 제1항, 제3...","유사수신행위의 규제에 관한 법률(이하 ‘유사수신행위법’이라 한다) 제6조 제1항, ...","유사수신행위의 규제에 관한 법률 제3조, 제6조 제1항, 형법 제37조, 제347조...",내용없음,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 장용배【배상신청인】 배...


# 0. 사건 Input

In [3]:
user_input = "저는 3대 독자인 남편과 혼인하여 3명의 딸을 두고 있어 아들 낳기를 고심하던 중 甲이 운영하는 병원에서 시술을 받으면\
    아들을 낳을 수 있다는 소문을 들었습니다. 저는 甲의 병원에 찾아가 甲에게 “아들을 낳고 싶다.”라고 하면서 甲으로부터 \
    일련의 시술과 처방을 받고 의료수가 및 약값 명목으로 금원을 지불하였으나 아무런 효과가 없었습니다. \
    이 경우 甲에게 형사상 죄책을 물을 수 있는지요?"

In [4]:
import hanja
import re
from nltk import sent_tokenize


def prep(sent):
    sent = hanja.translate(sent, 'substitution')
    sent = re.sub(r'\?', '.', sent)
    sent = re.sub('[^가-힣. ]', '', sent)

    return sent

def tokenize(sent):
    sent_tokened = sent_tokenize(sent)

    return sent_tokened

In [5]:
input_prep = prep(user_input)
input_tokened = tokenize(input_prep)
input_tokened

['저는 대 독자인 남편과 혼인하여 명의 딸을 두고 있어 아들 낳기를 고심하던 중 갑이 운영하는 병원에서 시술을 받으면    아들을 낳을 수 있다는 소문을 들었습니다.',
 '저는 갑의 병원에 찾아가 갑에게 아들을 낳고 싶다.라고 하면서 갑으로부터     일련의 시술과 처방을 받고 의료수가 및 약값 명목으로 금원을 지불하였으나 아무런 효과가 없었습니다.',
 '이 경우 갑에게 형사상 죄책을 물을 수 있는지요.']

# 1. FastText

### 1.1 모델 & 임베딩 로드

In [6]:
from gensim.models import FastText

# 모델 로드
fasttext = FastText.load('./model/fasttext.model')

# 임베딩 값 로드
with open("./model/fasttext_emb.pickle", 'rb') as fr:
    fasttext_embedding = pk.load(fr)

### 1.2 사용자 입력 임베딩

In [7]:
embedding = None
count = 0
for sentence in input_tokened:
    if embedding is None:
        embedding = fasttext.wv.get_sentence_vector(sentence)
    else:
        embedding += fasttext.wv.get_sentence_vector(sentence)
    count += 1

f_embedding = embedding / count

### 1.3 코사인 유사도 계산

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

f_similarity = cosine_similarity([f_embedding], fasttext_embedding)
f_similarity = list(enumerate(f_similarity[0]))
f_similarity = sorted(f_similarity, key=lambda x : x[1], reverse = True)

### 1.4 유사 판례 추출

In [9]:
precidents = []

for i in range(5):
    idx = f_similarity[i][0]
    sim = round(f_similarity[i][1] * 100, 2)
    
    precidents.append([case_fraud['판례일련번호'][idx], case_fraud['사건명'][idx], case_fraud['판례내용'][idx], sim])

In [10]:
f_result = pd.DataFrame(precidents, columns=['판례일련번호', '사건명', '판례내용', '유사도'])
f_result

,판례일련번호,사건명,판례내용,유사도
0,183664,사기·의료법위반,【피 고 인】 【상 고 인】 피고인들【변 호 인】 변호사 조재현 외 1인【원심판결】...,95.36
1,64755,특정경제범죄가중처벌등에관한법률위반(사기)·사문서위조·위조사문서행사·사기·무고,【피 고 인】 【상 고 인】 피고인【변 호 인】 변호사 박중욱【원심판결】 대전고법 ...,95.10
2,105837,사기,【피 고 인】 【상 고 인】 검사【원심판결】 \n서울형사지방법원 1989.3.1...,94.85
3,206151,특정경제범죄가중처벌등에관한법률위반(사기)·유가증권위조(일부인정된죄명유가증권변조)·위...,"【피 고 인】 피고인 1 외 2인【항 소 인】 검사 및 (피고인 1, 피고인 2에 ...",94.78
4,204534,사기,"【피 고 인】 【항 소 인】 검사【검 사】 이유현(기소), 이윤환(공판)【원심...",94.74


In [11]:
f_result.판례내용[0]

'【피 고 인】 【상 고 인】 피고인들【변 호 인】 변호사 조재현 외 1인【원심판결】 수원지법 2016. 7. 1. 선고 2016노2573 판결【주    문】  상고를 모두 기각한다.【이    유】  상고이유를 판단한다.  1. 피고인 1의 상고이유에 관하여  원심판결에 양형사유에 관한 심리미진의 위법이 있다는 취지의 주장은 결국 양형부당 주장에 해당한다. 그런데 형사소송법 제383조 제4호에 의하면 사형, 무기 또는 10년 이상의 징역이나 금고가 선고된 사건에서만 양형부당을 사유로 한 상고가 허용되므로, 피고인에 대하여 그보다 가벼운 형이 선고된 이 사건에서 형의 양정이 부당하다는 취지의 주장은 적법한 상고이유가 되지 못한다.  2. 피고인 2의 상고이유에 관하여  가. 의료법 제33조 제8항에서 의사가 개설·운영할 수 있는 의료기관의 수를 1개소로 제한하고 있는 취지는 의료기관을 개설하는 의사가 자신의 면허를 바탕으로 개설된 의료기관에서 이루어지는 의료행위에 전념하도록 하기 위하여 장소적 한계를 설정함으로써 의료의 적정을 기하여 국민의 건강을 보호·증진하고자 하는 데 있다.따라서 이미 자신 명의로 의료기관을 개설·운영하면서 의료행위를 하고 있는 의사가 다른 의사를 고용하여 그 의사 명의로 새로운 의료기관을 개설하고 그 운영에 직접 관여하는 데서 더 나아가 그 의료기관에서 자신이 직접 의료행위를 하거나 비의료인을 고용하여 자신의 주관하에 의료행위를 하게 한 경우에는 이미 자신의 명의로 의료기관을 개설·운영하고 있는 위 의사로서는 중복하여 의료기관을 개설한 경우에 해당한다(대법원 2003. 10. 23. 선고 2003도256 판결, 대법원 2008. 9. 25. 선고 2006도4652 판결 등 참조). 또한 이미 자신의 명의로 의료기관을 개설·운영하면서 의료행위를 하고 있는 의사가 다른 의사가 개설·운영하고 있는 기존 의료기관을 인수하여 의료법 제33조 제5항 등에 따른 개설자 명의변경 신고 또는 허가를 받지 아니한 채 또는 다른 의사의 면허증을 대여받아 그 